<a href="https://colab.research.google.com/github/22203007/Bhuvan/blob/main/real_time_project%20for%20scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from openpyxl.styles import Font

# Send a GET request to the website
url = "https://www.omiclear.pt/en/dados-mercado?date=2024-05-23&product=NG&zone=ES&instrument=FGF"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the input tag with the date value
input_tag = soup.find('input', {'type': 'date'})

# Extract the date value from the input tag
date_value = input_tag['value']

# Find all tables on the page
tables = soup.find_all('table')

# Create a list to store the table data
table_data = []

# Loop through each table
for table in tables:
    # Extract the table headers
    headers = [th.text.strip() for th in table.find('tr').find_all('th')]

    # Extract the table data
    data = []
    for row in table.find_all('tr')[1:]:
        row_data = []
        for td in row.find_all('td'):
            # Extract the text outside the <span> tags
            text = td.text.strip()
            for span in td.find_all('span'):
                text = text.replace(span.text, '')
            row_data.append(text.strip())
        data.append(row_data)

    # Create a Pandas DataFrame from the table data
    # Use the first row as the column names
    df = pd.DataFrame(data[1:], columns=data[0])

    # Add the DataFrame to the list
    table_data.append(df)

# Concatenate the DataFrames
combined_df = pd.concat(table_data, ignore_index=True)

# Create an Excel writer
with pd.ExcelWriter(f'{date_value}.xlsx') as writer:
    # Write the combined DataFrame to a single sheet
    combined_df.to_excel(writer, sheet_name='FGF', index=False, header=True)

    # Get the workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['FGF']

    # Set the format for the header row
    for cell in worksheet[1]:
        cell.font = Font(bold=True)

print("Data saved to", date_value, ".xlsx")

Data saved to 2024-05-23 .xlsx
